In [24]:
!pip install groq python-dotenv numpy tqdm datasets

In [25]:
from groq import Groq
from dotenv import load_dotenv
from datasets import load_dataset

import os
from tqdm import tqdm
import re
import random
import pprint

from typing import List, Dict, Any

load_dotenv()
random.seed(0)

client = Groq()
gsm8k_dataset = load_dataset("gsm8k", "main")

gsm8k_train = gsm8k_dataset["train"]
gsm8k_test  = gsm8k_dataset["test"]

In [26]:
def generate_response_using_Llama(
        prompt: str,
        model: str = "llama3-8b-8192"
    ):
    try:
        chat_completion = client.chat.completions.create(
            messages=[
                {
                    "role": "system",
                    "content": "You are a helpful assistant that solves math problems."
                },
                {
                    "role": "user", 
                    "content": prompt
                }
            ],
            model=model,
            temperature=0.3, ### 수정해도 됩니다!
            stream=False
        )
        return chat_completion.choices[0].message.content
    
    except Exception as e:
        print(f"API call error: {str(e)}")
        return None

#### 응답 잘 나오는지 확인해보기

In [27]:
response = generate_response_using_Llama(
    prompt="Hello world!",
)
print(response)

Hello! I'm excited to help you with your math problems. What kind of math do you need help with? Do you have a specific problem you're stuck on or a concept you're struggling with? Let me know and I'll do my best to assist you!


#### GSM8K 데이터셋 확인해보기

In [28]:
print("[Question]")
for l in gsm8k_test['question'][0].split("."):
    print(l)
print("="*100)
print("[Answer]")
print(gsm8k_test['answer'][0])

[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18


#### Util 함수들
- extract_final_answer: LLM의 응답을 parse하여 최종 결과만 추출 (정답과 비교하기 위해)
- run_benchmark_test: 벤치마크 테스트
- save_final_result: 결과물 제출을 위한 함수

In [29]:
### 수정해도 됩니다!
def extract_final_answer(response: str):
    regex = r"(?:Answer:|Model response:)\s*\$?([0-9,]+)\b|([0-9,]+)\s*(meters|cups|miles|minutes)"
    matches = re.finditer(regex, response, re.MULTILINE)
    results = [match.group(1) if match.group(1) else match.group(2).replace(",", "") for match in matches]

    if len(results) == 0:
        additional_regex = r"\$?([0-9,]+)"
        additional_matches = re.finditer(additional_regex, response, re.MULTILINE)
        results.extend([match.group(1).replace(",", "") for match in additional_matches])

    return results[-1] if results else None

In [30]:
### 수정해도 됩니다!
def run_benchmark_test(
        dataset,
        prompt: str,
        model: str = "llama3-8b-8192",
        num_samples: int = 50,
        VERBOSE: bool = False
    ):
    correct = 0
    total   = 0
    results = []

    for i in tqdm(range(min(num_samples, len(dataset)))):
        question = dataset[i]["question"]
        correct_answer = float(re.findall(r'\d+(?:\.\d+)?', dataset[i]["answer"].split('####')[-1])[0])

        response = generate_response_using_Llama(
            prompt=prompt.format(question=question),
            model=model
        )

        if response:
            if VERBOSE:
                print("="*50)
                print(response)
                print("="*50)
            predicted_answer = extract_final_answer(response)

            if isinstance(predicted_answer, str):
                if predicted_answer.strip() == "":
                    predicted_answer = None
                else:
                    try:
                        predicted_answer = float(predicted_answer.replace(",", ""))
                    except ValueError:
                        predicted_answer = None
                    if predicted_answer is not None:
                        diff = abs(predicted_answer - correct_answer)
                        is_correct = diff < 1e-5
                    else:
                        is_correct = False
                                
            is_correct = diff < 1e-5 if predicted_answer is not None else False
            
            if is_correct:
                correct += 1
            total += 1
            
            results.append({
                'question': question,
                'correct_answer': correct_answer,
                'predicted_answer': predicted_answer,
                'response': response,
                'correct': is_correct
            })

            if (i + 1) % 5 == 0:
                current_acc = correct/total if total > 0 else 0
                print(f"Progress: [{i+1}/{num_samples}]")
                print(f"Current Acc.: [{current_acc:.2%}]")

    return results, correct/total if total > 0 else 0

In [31]:
def save_final_result(results: List[Dict[str, Any]], accuracy: float, filename: str) -> None:
    result_str = f"====== ACCURACY: {accuracy} ======\n\n"
    result_str += f"[Details]\n"
    
    for idx, result in enumerate(results):
        result_str += f"Question {idx+1}: {result['question']}\n"
        result_str += f"Correct Answer: {result['correct_answer']}\n"
        result_str += f"Predicted Answer: {result['predicted_answer']}\n"
        result_str += f"Correct: {result['correct']}\n\n"
    
    with open(filename, "w", encoding="utf-8") as f:
        f.write(result_str)

#### Direct prompting with few-shot example

In [32]:
def construct_direct_prompt(num_examples: int = 3) -> str:
    train_dataset = gsm8k_train

    sampled_indices = random.sample(
        [i for i in range(len(train_dataset['question']))],
        num_examples
    )

    prompt = "Instruction:\nSolve the following mathematical question and generate ONLY the answer after a tag, 'Answer:' without any rationale.\n"

    for i in range(num_examples):
        cur_question = train_dataset['question'][i]
        cur_answer = train_dataset['answer'][i].split("####")[-1].strip()

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"

    return prompt

In [33]:
### 어떤 방식으로 저장되는지 확인해보세요!
PROMPT = construct_direct_prompt(3)
VERBOSE = False

results, accuracy = run_benchmark_test(
    dataset=gsm8k_test,
    prompt=PROMPT,
    VERBOSE=VERBOSE,
    num_samples=10
)
save_final_result(results, accuracy, "example.txt")

 50%|█████     | 5/10 [00:03<00:03,  1.49it/s]

Progress: [5/10]
Current Acc.: [20.00%]


100%|██████████| 10/10 [00:06<00:00,  1.65it/s]

Progress: [10/10]
Current Acc.: [30.00%]


In [34]:
# TODO: 0 shot, 3 shot, 5 shot direct prompting을 통해 벤치마크 테스트를 한 후, 각각 direct_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 direct_prompting_5.txt
# 항상 num_samples=50 입니다!

for shot in [0, 3, 5]:
    PROMPT = construct_direct_prompt(shot)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        VERBOSE=VERBOSE,
        num_samples=50
    )
    save_final_result(results, accuracy, f"direct_prompting_{shot}.txt")
    print(f"{shot} shot completed with {accuracy:.2%} accuracy")

 10%|█         | 5/50 [00:02<00:24,  1.86it/s]

Progress: [5/50]
Current Acc.: [20.00%]


 20%|██        | 10/50 [00:05<00:21,  1.89it/s]

Progress: [10/50]
Current Acc.: [10.00%]


 30%|███       | 15/50 [00:07<00:19,  1.79it/s]

...

Progress: [40/50]
Current Acc.: [17.50%]


 90%|█████████ | 45/50 [01:57<00:12,  2.52s/it]

Progress: [45/50]
Current Acc.: [15.56%]


100%|██████████| 50/50 [02:12<00:00,  2.65s/it]

Progress: [50/50]
Current Acc.: [18.00%]
5 shot completed with 18.00% accuracy


### Chain-of-Thought prompting with few-shot example
```text
[Question]
Janet’s ducks lay 16 eggs per day
 She eats three for breakfast every morning and bakes muffins for her friends every day with four
 She sells the remainder at the farmers' market daily for $2 per fresh duck egg
 How much in dollars does she make every day at the farmers' market?
====================================================================================================
[Answer]
Janet sells 16 - 3 - 4 = <<16-3-4=9>>9 duck eggs a day.
She makes 9 * 2 = $<<9*2=18>>18 every day at the farmer’s market.
#### 18
```

[Answer] 아래의 정답을 도출하는 과정을 예시로 달아주면 CoT의 few shot이 되겠죠?

In [35]:
def construct_CoT_prompt(num_examples: int = 3) -> str:
    """CoT 프롬프트를 구성하는 함수"""
    train_dataset = gsm8k_train
    
    if num_examples > len(train_dataset):
        num_examples = len(train_dataset)

    sampled_indices = random.sample(
        range(len(train_dataset)),
        num_examples
    )
    
    prompt = ("Instruction:\n" 
              "Solve the following mathematical question. Provide a step-by-step rationale for your answer, " 
              "and then provide the final answer in the format '#### <answer>'.\n")

    for i, idx in enumerate(sampled_indices):
        cur_question = train_dataset['question'][idx]
        cur_answer = train_dataset['answer'][idx]

        prompt += f"\n[Example {i+1}]\n"
        prompt += f"Question:\n{cur_question}\n"
        prompt += f"Answer:\n{cur_answer}\n"

    prompt += "\nQuestion:\n{question}\nAnswer:"
    
    return prompt


In [36]:
# TODO: 0 shot, 3 shot, 5 shot CoT prompting을 통해 벤치마크 테스트를 한 후, 각각 CoT_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 CoT_prompting_5.txt
# 항상 num_samples=50 입니다!

for shot in [0, 3, 5]:
    PROMPT = construct_CoT_prompt(shot)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        VERBOSE=VERBOSE,
        num_samples=50
    )
    save_final_result(results, accuracy, f"CoT_prompting_{shot}.txt")
    print(f"{shot} shot completed with {accuracy:.2%} accuracy")

 10%|█         | 5/50 [00:13<01:59,  2.65s/it]

Progress: [5/50]
Current Acc.: [80.00%]


 20%|██        | 10/50 [00:32<02:20,  3.50s/it]

Progress: [10/50]
Current Acc.: [70.00%]


 30%|███       | 15/50 [00:51<02:17,  3.92s/it]

...

Progress: [40/50]
Current Acc.: [75.00%]


 90%|█████████ | 45/50 [09:46<01:04, 12.81s/it]

Progress: [45/50]
Current Acc.: [75.56%]


100%|██████████| 50/50 [10:51<00:00, 13.03s/it]

Progress: [50/50]
Current Acc.: [74.00%]
5 shot completed with 74.00% accuracy


### Construct your prompt!!

목표: 본인만의 프롬프트를 통해 정답률을 더 끌어올려보기!
- gsm8k의 train 데이터셋에서 예시를 가져온 다음 (자유롭게!)
- 그 예시들에 대한 풀이 과정을 만들어주세요!
- 모든 것들이 자유입니다! Direct Prompting, CoT Prompting을 한 결과보다 정답률만 높으면 돼요.

In [55]:
example_list = [
    """Question:
A bag has 5 red balls and 3 blue balls. If you take out 2 red balls, how many red balls remain?
Scratchpad:
Start with 5 red balls.
Take out 2 → 5 - 2 = 3 red balls left.
Answer: 3""",

    """Question:
A car travels 60 km in 1.5 hours. What is its average speed in km/h?
Scratchpad:
Distance = 60 km, Time = 1.5 h
Speed = Distance / Time = 60 / 1.5 = 40 km/h
Answer: 40""",

    """Question:
A store offers a 25% discount on a $200 item. What is the final price?
Scratchpad:
Discount = 25% of 200 = 0.25 × 200 = 50
Final price = 200 - 50 = 150
Answer: 150"""
]

from typing import List
import re
import time

# -------------------------
# 1. Scratchpad Prompt 생성
# -------------------------
def construct_my_prompt(example_list: List[str], num_examples: int = 3):
    prompt = (
        "Instruction:\n"
        "You are a careful math tutor. Solve each math problem step-by-step.\n"
        "Rules:\n"
        "1. Restate the problem briefly.\n"
        "2. List all known values.\n"
        "3. Show every calculation step explicitly.\n"
        "4. Keep intermediate numbers consistent.\n"
        "5. At the end, write ONLY the final number after 'Answer:'. No units, no extra words.\n"
    )
    
    for i, example in enumerate(example_list[:num_examples]):
        prompt += f"\n[Example {i+1}]\n{example.strip()}\n"

    prompt += "\nQuestion:\n{question}\nScratchpad:\nAnswer:"
    return prompt


# -------------------------
# 2. 답 추출 개선
# -------------------------
def extract_answer(text: str) -> str:
    # 'Answer:' 또는 'Correct Answer:' 뒤의 숫자를 우선 추출
    match = re.search(r"(?:Answer|Correct Answer)\s*[:\-]?\s*(-?\d+(?:\.\d+)?)", text, re.IGNORECASE)
    if match:
        return match.group(1)
    # 없으면 마지막 숫자를 추출
    matches = re.findall(r"-?\d+(?:\.\d+)?", text)
    return matches[-1] if matches else "N/A"


# -------------------------
# 3. 실행 함수 (강화된 검증)
# -------------------------
def run_with_answer_verification(prompt_template, dataset, num_samples=10):
    results = []
    correct = 0

    for i in range(num_samples):
        q = dataset['question'][i]
        gt = dataset['answer'][i].split("####")[-1].strip()

        # 1차 추론
        reasoning_prompt = prompt_template.format(question=q)
        reasoning_output = generate_response_using_Llama(reasoning_prompt)

        # 2차 검증 - 무조건 새로 풀기
        verification_prompt = (
            "Instruction:\n"
            "The following math problem was solved, but the answer might be wrong.\n"
            "Ignore the previous solution completely. Solve from scratch.\n"
            "Follow the same rules:\n"
            " - Restate briefly, list values, calculate step-by-step.\n"
            " - End with ONLY the number after 'Correct Answer:'.\n\n"
            f"Original Question:\n{q}\n\n"
            "Correct Answer:"
        )
        verification_output = generate_response_using_Llama(verification_prompt)

        # 최종 답 추출
        verified_answer = extract_answer(verification_output)

        try:
            is_correct = abs(float(verified_answer) - float(gt)) < 1e-3
        except:
            is_correct = False

        correct += is_correct

        results.append({
            "question": q,
            "correct_answer": gt,
            "predicted_answer": verified_answer,
            "correct": is_correct
        })

        if (i + 1) % 5 == 0:
            current_acc = correct / (i + 1)
            print(f"Progress: [{i+1}/{num_samples}] | Current Acc.: {current_acc:.2%}")

    accuracy = correct / num_samples
    return results, accuracy


# -------------------------
# 프롬프트 만들기
# -------------------------
prompt_template = construct_my_prompt(example_list, num_examples=3)


In [56]:
# TODO: 만든 0 shot, 3 shot, 5 shot example과 프롬프트를 통해 벤치마크 테스트를 한 후, 각각 My_prompting_{shot: int}.txt로 저장해주세요!
# 예시: shot이 5인 경우 My_prompting_5.txt
# 항상 num_samples=50 입니다!

for shot in [0, 3, 5]:
    PROMPT = construct_my_prompt(gsm8k_train, num_examples=shot)
    results, accuracy = run_benchmark_test(
        dataset=gsm8k_test,
        prompt=PROMPT,
        VERBOSE=VERBOSE,
        num_samples=50
    )
    save_final_result(results, accuracy, f"My_prompting_{shot}.txt")
    print(f"{shot} shot completed with {accuracy:.2%} accuracy")

  6%|▌         | 3/50 [00:11<02:06,  2.69s/it]

API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01k1r7amteerjtz9y66485vm8m` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500231, Requested 147. Please try again in 1m5.489999999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01k1r7amteerjtz9y66485vm8m` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500230, Requested 166. Please try again in 1m8.6002s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


 10%|█         | 5/50 [00:12<00:51,  1.15s/it]

API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01k1r7amteerjtz9y66485vm8m` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500230, Requested 151. Please try again in 1m5.845199999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01k1r7amteerjtz9y66485vm8m` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500229, Requested 239. Please try again in 1m20.8916s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


 14%|█▍        | 7/50 [00:12<00:26,  1.65it/s]

...

API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01k1r7amteerjtz9y66485vm8m` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500198, Requested 149. Please try again in 1m0.0986s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01k1r7amteerjtz9y66485vm8m` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500197, Requested 159. Please try again in 1m1.6666s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


 74%|███████▍  | 37/50 [00:17<00:02,  5.96it/s]

API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01k1r7amteerjtz9y66485vm8m` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500196, Requested 164. Please try again in 1m2.363599999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}
API call error: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-8b-8192` in organization `org_01k1r7amteerjtz9y66485vm8m` service tier `on_demand` on tokens per day (TPD): Limit 500000, Used 500196, Requested 153. Please try again in 1m0.3088s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': 'tokens', 'code': 'rate_limit_exceeded'}}


 74%|███████▍  | 37/50 [00:17<00:06,  2.06it/s]


KeyboardInterrupt: 

### 보고서 작성하기
#### 아래의 내용이 포함되면 됩니다!

1. Direct Prompting, CoT Prompting, My Prompting을 0 shot, 3 shot, 5 shot 정답률을 표로 보여주세요!
2. CoT Prompting이 Direct Prompting에 비해 왜 좋을 수 있는지에 대해서 서술해주세요!
3. 본인이 작성한 프롬프트 기법이 CoT에 비해서 왜 더 좋을 수 있는지에 대해서 설명해주세요!
4. 최종적으로, `PROMPTING.md`에 보고서를 작성해주세요!